In [2]:
import $ivy.`org.apache.spark::spark-sql:3.2.2`
import $ivy.`org.apache.spark::spark-core:3.2.2`
import $ivy.`org.apache.spark::spark-mllib:3.2.2`
import $ivy.`org.tallison:jmatio:1.3`


import $ivy.$                                  

import $ivy.$                                   

import $ivy.$                                    

import $ivy.$                        


In [30]:
import org.apache.log4j.{Level, Logger}
import org.apache.log4j.{Level, Logger}
import org.apache.spark.sql._
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.regression._
import org.apache.spark.mllib.classification.{SVMModel,
SVMWithSGD}
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.ml.feature.LabeledPoint
import org.apache.spark.ml.linalg.Vectors
import com.jmatio._
import com.jmatio.types._
import org.apache.spark.ml.feature.PCA
import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.mllib.linalg.distributed
import org.apache.spark.mllib.linalg.distributed.RowMatrix
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
def sc = spark.sparkContext

Creating SparkSession


Spark UI

import org.apache.log4j.{Level, Logger}

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import org.apache.spark.mllib.linalg._

import org.apache.spark.mllib.regression._

import org.apache.spark.mllib.classification.{SVMModel,
SVMWithSGD}

import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics

import org.apache.spark.mllib.util.MLUtils

import org.apache.spark.ml.feature.LabeledPoint

import org.apache.spark.ml.linalg.Vectors

import com.jmatio._

import com.jmatio.types._

import org.apache.spark.ml.feature.PCA

import org.apache.spark.ml.linalg.Matrix

import org.apache.spark.mllib.linalg.distributed

import org.apache.spark.mllib.linalg.distributed.RowMatrix

spark: SparkSession = org.apache.spark.sql.SparkSession@3f43b89e
defined function sc

In [7]:
val df = new com.jmatio.io.MatFileReader("/home/jb-admin/data/exPCA.mat")

df: io.MatFileReader = com.jmatio.io.MatFileReader@4415979a

In [8]:
val content = df.getContent

content: java.util.Map[String, com.jmatio.types.MLArray] = {X=[50x2  double array]}

In [10]:
val data = content.get("X").asInstanceOf[MLDouble].getArray

data: Array[Array[Double]] = Array(
  Array(3.38156266663556, 3.389112684892071),
  Array(4.5278753804032075, 5.854178101169413),
  Array(2.655681868739463, 4.411994717484785),
  Array(2.7652346742250753, 3.715413649743292),
  Array(2.8465601062210943, 4.175506449514385),
  Array(3.8906719563092116, 6.488380871886209),
  Array(3.4758052414407934, 3.632848762047064),
  Array(5.911298445495829, 6.6807685267677925),
  Array(3.928893967969269, 5.0984466081478255),
  Array(4.561835366089421, 5.623299290382872),
  Array(4.574071705525158, 5.3976506891499545),
  Array(4.371733557330694, 5.461165489180036),
  Array(4.191693876250999, 4.954693590451864),
  Array(5.244085176866642, 4.661487668490745),
  Array(2.8358402028078746, 3.76801716326883),
  Array(5.635269692588768, 6.3121143831055955),
  Array(4.686329679649661, 5.6652411030489915),
  Array(2.8505133748624067, 4.626456272707626),
  Array(5.110157300375674, 7.3631966235366235),
  Array(5.182563768446949, 4.646509087781815),
  Array(5.707

In [25]:
val numFeatures = data(0).length

// Calculate means for each feature
// Calculate means for each feature separately
val mean1: Double = data.map(_(0)).sum / data.length.toDouble
val mean2: Double = data.map(_(1)).sum / data.length.toDouble

// Subtract means from each value and create new array
val centeredData: Array[Array[Double]] = data.map { row =>
  Array(row(0) - mean1, row(1) - mean2)
}

// Print results
println("Original data:")
data.foreach(row => println(row.mkString(", ")))
println()
println("Means:")
println(means.mkString(", "))
println()
println("Centered data:")
centeredData.foreach(row => println(row.mkString(", ")))

Original data:
3.38156266663556, 3.389112684892071
4.5278753804032075, 5.854178101169413
2.655681868739463, 4.411994717484785
2.7652346742250753, 3.715413649743292
2.8465601062210943, 4.175506449514385
3.8906719563092116, 6.488380871886209
3.4758052414407934, 3.632848762047064
5.911298445495829, 6.6807685267677925
3.928893967969269, 5.0984466081478255
4.561835366089421, 5.623299290382872
4.574071705525158, 5.3976506891499545
4.371733557330694, 5.461165489180036
4.191693876250999, 4.954693590451864
5.244085176866642, 4.661487668490745
2.8358402028078746, 3.76801716326883
5.635269692588768, 6.3121143831055955
4.686329679649661, 5.6652411030489915
2.8505133748624067, 4.626456272707626
5.110157300375674, 7.3631966235366235
5.182563768446949, 4.646509087781815
5.707328091354587, 6.681039949775041
3.5796845825157533, 4.802780735462661
5.639377731233365, 6.120435944864187
4.263468511601602, 4.6894289649837795
2.5365169312575038, 3.884490775756531
3.223829017502572, 4.942555853672868
4.9294880

numFeatures: Int = 2
mean1: Double = 3.9892652815843155
mean2: Double = 5.002805850257663
centeredData: Array[Array[Double]] = Array(
  Array(-0.6077026149487557, -1.613693165365592),
  Array(0.538610098818892, 0.8513722509117505),
  Array(-1.3335834128448525, -0.5908111327728776),
  Array(-1.2240306073592402, -1.287392200514371),
  Array(-1.1427051753632211, -0.8272994007432777),
  Array(-0.0985933252751039, 1.485575021628546),
  Array(-0.5134600401435221, -1.3699570882105987),
  Array(1.9220331639115136, 1.6779626765101296),
  Array(-0.06037131361504633, 0.09564075789016258),
  Array(0.5725700845051058, 0.6204934401252089),
  Array(0.5848064239408428, 0.3948448388922916),
  Array(0.38246827574637887, 0.4583596389223734),
  Array(0.20242859466668328, -0.04811225980579881),
  Array(1.2548198952823268, -0.3413181817669182),
  Array(-1.1534250787764408, -1.234788686988833),
  Array(1.6460044110044523, 1.3093085328479326),
  Array(0.6970643980653453, 0.6624352527913286),
  Array(-1.138751

In [28]:
val vectors = centeredData.map(row => Vectors.dense(row))
// print the resulting vectors
vectors.foreach(println)

[-0.6077026149487557,-1.613693165365592]
[0.538610098818892,0.8513722509117505]
[-1.3335834128448525,-0.5908111327728776]
[-1.2240306073592402,-1.287392200514371]
[-1.1427051753632211,-0.8272994007432777]
[-0.0985933252751039,1.485575021628546]
[-0.5134600401435221,-1.3699570882105987]
[1.9220331639115136,1.6779626765101296]
[-0.06037131361504633,0.09564075789016258]
[0.5725700845051058,0.6204934401252089]
[0.5848064239408428,0.3948448388922916]
[0.38246827574637887,0.4583596389223734]
[0.20242859466668328,-0.04811225980579881]
[1.2548198952823268,-0.3413181817669182]
[-1.1534250787764408,-1.234788686988833]
[1.6460044110044523,1.3093085328479326]
[0.6970643980653453,0.6624352527913286]
[-1.1387519067219087,-0.3763495775500365]
[1.1208920187913582,2.3603907732789606]
[1.1932984868626333,-0.3562967624758482]
[1.718062809770272,1.6782340995173781]
[-0.4095806990685622,-0.20002511479500207]
[1.6501124496490496,1.1176300946065245]
[0.2742032300172861,-0.3133768852738834]
[-1.45274835032681

vectors: Array[Vector] = Array(
  [-0.6077026149487557,-1.613693165365592],
  [0.538610098818892,0.8513722509117505],
  [-1.3335834128448525,-0.5908111327728776],
  [-1.2240306073592402,-1.287392200514371],
  [-1.1427051753632211,-0.8272994007432777],
  [-0.0985933252751039,1.485575021628546],
  [-0.5134600401435221,-1.3699570882105987],
  [1.9220331639115136,1.6779626765101296],
  [-0.06037131361504633,0.09564075789016258],
  [0.5725700845051058,0.6204934401252089],
  [0.5848064239408428,0.3948448388922916],
  [0.38246827574637887,0.4583596389223734],
  [0.20242859466668328,-0.04811225980579881],
  [1.2548198952823268,-0.3413181817669182],
  [-1.1534250787764408,-1.234788686988833],
  [1.6460044110044523,1.3093085328479326],
  [0.6970643980653453,0.6624352527913286],
  [-1.1387519067219087,-0.3763495775500365],
  [1.1208920187913582,2.3603907732789606],
  [1.1932984868626333,-0.3562967624758482],
  [1.718062809770272,1.6782340995173781],
  [-0.4095806990685622,-0.20002511479500207],
 

In [29]:
val rows = sc.parallelize(vectors)

rows: org.apache.spark.rdd.RDD[Vector] = ParallelCollectionRDD[1] at parallelize at cmd28.sc:1

In [31]:
val mat: RowMatrix = new RowMatrix(rows)

mat: RowMatrix = org.apache.spark.mllib.linalg.distributed.RowMatrix@c7b065

In [34]:
val pca: Matrix = mat.computePrincipalComponents(1)

treeAggregate at Statistics.scala:58

12 / 12

treeAggregate at Statistics.scala:58

12 / 12

treeAggregate at Statistics.scala:58

...

treeAggregate at RowMatrix.scala:156

3 / 3

treeAggregate at RowMatrix.scala:156

3 / 3

pca: Matrix = -0.76908153413682    
-0.6391506816469457  

In [36]:
val projected: RowMatrix = mat.multiply(pca)

projected: RowMatrix = org.apache.spark.mllib.linalg.distributed.RowMatrix@3334fe61

In [41]:
projected.rows.take(50).foreach(row => println(row.toArray.head))

take at cmd40.sc:1

1 / 1

take at cmd40.sc:1

1 / 1

take at cmd40.sc:1

...

take at cmd40.sc:1

7 / 7

take at cmd40.sc:1

7 / 7

1.4987659460161817
-0.9583902356067575
1.403251715286524
1.7642169398439895
1.4076024252456056
-0.8736799818533338
1.270501642148402
-2.5506712028324956
-0.01469839310581298
-0.836941884205521
-0.7021291696202776
-0.5871101639263723
-0.12493311048896202
-0.7469050615943139
1.676293960076117
-2.1027570388671952
-0.9594953000247992
1.1163371524515604
-2.370702725108184
-0.6900165123544217
-2.3939748501512166
0.44284714086021515
-1.9834050510712578
-0.010589590971766455
1.832053772147532
0.6271917226503412
-1.3317160771397554
-1.454672696435523
1.019190975397314
0.01489202388535582
-0.07212621590129081
-1.7853951282973
1.4131805092372387
-0.8264452277798662
0.7516737667923108
-1.4055108052784504
1.82309802226081
-1.594588411978099
2.807836134039267
-0.3255152736303091
-0.985787622007345
0.9846546899375344
1.3895283574490214
-1.0374206198024047
1.8768659729028192
-0.24535116665387552
3.518002175282364
1.5486044117786215
0.34412681544995294
-1.5597867544521047
